In [5]:
import pandas as pd
import numpy as np
from yelpapi import YelpAPI
api_key = 'NmaOvYquqktGmkmomZ6_9_SUSvMFfOqsr_YHl_mOMrs8Hph0FnPr_ennDAvviX-Vn87J71li6WWNx3XwAeFrdwQzgsNcElpL3L7zi8bWuhdVI2FkGMEbZDukioCvXnYx'
yelp_api = YelpAPI(api_key)

In [6]:
dohmh_property = pd.read_csv('dohmh_property.csv')
unique_zipcode = np.load('unique_zipcode.npy')

In [7]:
def search_query(zip_codes):
    df = pd.DataFrame(columns=['name', 'cuisine', 'is_closed', 'review_count',
                               'rating', 'price', 'address','zip_code', 'latitude','longitude','phone'])
    for zip_code in zip_codes:
        try:
            response = yelp_api.search_query(location=str(zip_code), sort_by='distance', limit=50)['businesses']
        except YelpAPI.YelpAPIError:
            pass
            
        for i in range(len(response)):
            if response[i]['location']['zip_code'] == str(zip_code):
                try:
                    df = df.append({'name': response[i]['name'],
                                        'cuisine': response[i]['categories'][0]['alias'],
                                        'is_closed': response[i]['is_closed'],
                                        'review_count': response[i]['review_count'],
                                        'rating': response[i]['rating'],
                                        'price': response[i]['price'],
                                        'address': response[i]['location']['address1'],
                                        'zip_code': response[i]['location']['zip_code'],
                                        'latitude':response[i]['coordinates']['latitude'],
                                        'longitude': response[i]['coordinates']['longitude'],
                                        'phone': response[i]['phone']}, 
                                           ignore_index=True)
                except:
                    pass
    return df


In [ ]:
unique_zipcodes = pd.unique(restaurants.zipcode)
# yelp_restaurants = search_query(unique_zipcodes)
# yelp_restaurants.to_csv('yelp_nyc_restaurants.csv')

In [8]:
def join_yelp_dohmh_property(dohmh_property, yelp_rest):
    dohmh_rest = dohmh_property[['DBA','CUISINE DESCRIPTION',  'SCORE', 'GRADE','BORO', 'BUILDING', 'STREET', 'ZIPCODE', 
                'CRITICAL FLAG','INSPECTION TYPE', 'VIOLATION CODE','ACTION','GRADE DATE','YEAR', 'AVERAGE PRICE']]
    yelp_rest['building'] = yelp_rest.address.str.extract(r'(\d+)')
    joined = pd.merge(dohmh_rest , yelp_rest, how = 'inner', left_on=['ZIPCODE', 'BUILDING'], right_on=['zip_code', 'building'])
    df = joined[[ # get name, cuisine type, inspection info, location from DOHMH
                'DBA','CUISINE DESCRIPTION', 'SCORE', 'GRADE','BORO', 'ZIPCODE', 
                'CRITICAL FLAG','INSPECTION TYPE', 'VIOLATION CODE','ACTION','GRADE DATE','YEAR', 
                 # get average property price from property
                'AVERAGE PRICE',
                # get location, number of reviews, rating, price from yelp
                'latitude', 'longitude', 'address', 'review_count', 'rating','price']]
    df['price'] = df.price.replace('$$$$', 4).replace('$$$', 3).replace('$$', 2).replace('$', 1)
    df = df.rename(columns = {'DBA': 'Name'})
    return  df

In [9]:
yelp_restaurants = pd.read_csv('yelp_nyc_restaurants.csv')
yelp_dohmh_property = join_yelp_dohmh_property(dohmh_property, yelp_restaurants)

/Users/aitingkuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [15]:
yelp_restaurants.cuisine.value_counts()

chinese          332
pizza            319
coffee           277
bakeries         212
delis            156
                ... 
landmarks          1
newmexican         1
nonprofit          1
armenian           1
healthmarkets      1
Name: cuisine, Length: 189, dtype: int64

In [18]:
dohmh_property['CUISINE DESCRIPTION'].value_counts()

American              31473
Chinese               18794
Caribbean              9357
Café/Coffee/Tea        8306
Pizza                  7047
                      ...  
Scandinavian             23
Southwestern             18
Iranian                  18
Nuts/Confectionary       15
Californian              12
Name: CUISINE DESCRIPTION, Length: 83, dtype: int64

In [ ]:
yelp_dohmh_property[['Name','SCORE','CUISINE DESCRIPTION','AVERAGE PRICE', 'review_count', 'rating', 'price']]